In [129]:
import numpy as np
from copy import deepcopy
from copy import copy
from functools import reduce
from itertools import product
import networkx as nx
from matplotlib import pyplot as plt
import sys
import networkplotter as plotter
import create_swot_table as swot
import pandas as pd

In [178]:
def form_SW_table(SWOT, t_count):
    sw = SWOT.index
    sw_table = np.zeros((len(sw), len(sw)))
    
    for i in range(len(sw) - 1):
        for j in range(i + 1, len(sw)):
            weight = 0
            for k in range(len(SWOT.iloc[i])):
                weight = weight - min(SWOT.iloc[i][k], SWOT.iloc[j][k]) if k < t_count \
                else weight + min(SWOT.iloc[i][k], SWOT.iloc[j][k])
           
            
            sw_table[i, j] = round(weight, 5)
    
    sw_table = sw_table + sw_table.T
    print(sw_table[0, :])
    SW = pd.DataFrame({SWOT.index[i]: sw_table[:, i] for i in range(sw_table.shape[1])}, index=SWOT.index)
    return SW
            

class Node(object):
    
    
    def __init__(self, name, value):
        self.name = name
        self.value = value
        self.connections = dict()
    
    
    def set_connections(self, connections):
        for connection in connections:
            self.connections[connection[1]] = connection[0]
      
    
    def set_weight(self, connection):
        self.connections[connection[1]] = connection[0]
    
    
    def set_value(self, value):
        self.value = value
    
    
    def remove_connection(self, node):
        try:
            self.connections.pop(node)
        except KeyError:
            print("No connection detected")
    
    
    def __str__(self):
        return self.name
    
    
class Graph(object):
    
    
    def __init__(self, name):
        self.name = name
        self.nodes = []
        self.A = []
    
    def from_pandas(self, SWOT, SW, s_count):
        nodes = dict()
        
        for index in SWOT.index:
            value = 0
            row = SWOT.loc[index]
            for i in range(len(row)):
                value = value + row[i]
            nodes[index] = Node(index, value)
            
        for column in SWOT.columns:
            value = 0
            col = SWOT[column]
            for i in range(len(col)):
                value = value + col[i] if i < s_count else value - col[i]
            nodes[column] = Node(column, value)
        
        for index in SWOT.index:
            for column in SWOT.columns:
                if SWOT.loc[index][column] != 0:
                    nodes[index].set_weight([SWOT.loc[index][column], nodes[column]])
                    nodes[column].set_weight([SWOT.loc[index][column], nodes[index]])
        
        for index in SW.index:
            for column in SW.columns:
                nodes[index].set_weight([SW.loc[index][column], nodes[column]]) \
                if SW.loc[index][column] != 0 else 0
                
        self.set_nodes(list(nodes.values()))
        
             
            
    
    def form_subgraphs(self, clusters):
        clusters_keys = list(clusters.keys())
        clusters_values  = list(clusters.values())
        subgraphs = []
        connections = []
        
        for cluster in clusters_keys:
            value = reduce(lambda x, y: x.value + y.value, clusters[cluster]) \
            if len(clusters[cluster]) > 1 else clusters[cluster][0].value 
            
            subgraphs.append(Node(cluster,value))    
        
        for i in range(len(clusters)):
            connections = []
            for j in range(len(clusters)):
                weight = 0
                for node_left in clusters_values[i]:
                    for node_right in node_left.connections.keys():
                        additive = node_left.connections[node_right] if node_right in clusters_values[j] \
                        and not node_right in clusters_values[i] else 0
                        weight += additive
                connections.append([weight, subgraphs[j]])
           
            subgraphs[i].set_connections(connections)
        
        aggregated = Graph('aggregated')
        aggregated.set_nodes(subgraphs)
        return aggregated
    
    
    def set_nodes(self, nodes):
        self.nodes = nodes
     
    
    def add_node(self, node):
        self.nodes.append(node)
        
        
    def remove_node(self, node):
        try:
            self.nodes.remove(node)
        except ValueError:
            print("no node " + str(node) + " in the graph")
    
    
    def __str__(self):
        return self.name
    
    
    def form_connection_matrix(self):
        self.A = np.zeros((len(self.nodes), len(self.nodes)))
        for i in range(len(self.nodes)):
            
            connection_nodes = list(self.nodes[i].connections.keys())
            for j in range(len(self.nodes)):
                self.A[i, j] = self.nodes[i].connections[self.nodes[j]] \
                if self.nodes[j] in connection_nodes else 0
        
        return self.A
    
    
    def update_values(self, nodes_values):
        for node in self.nodes:
            node.set_value(nodes_values[node])
    
    
    def send_impulses(self, impulses):
        previous = np.zeros((len(self.nodes), 1))
        current = np.array([[node.value] for node in self.nodes])
        for impulse in impulses:
            new = current + self.A.T @  (current - previous) + impulse
            nodes_values = {self.nodes[i]: current[i, 0]   for i in range(len(self.nodes))}
            previous = current
            current = new
            self.update_values(nodes_values)
        
        print(current)
        
    def search_cycles(self, verbose = False):
        cycles = set()
        for node in self.nodes:
            layer = 0
            vertexes = dict()
            vertexes[layer] = set([node])
            while True:
                nexts = set()
                previous_layers = reduce(lambda x, y: x.union(y), list(vertexes.values())[: layer]) if layer > 0 else []
                for vertex in vertexes[layer]:
                    if vertex not in previous_layers:
                        for next_vertex in vertex.connections.keys():
                            nexts.add(next_vertex)
                
                layer += 1
                if nexts != set():
                    vertexes[layer] = nexts
                else:
                    break  
            
            for length in range(1, len(vertexes)):
                if node in vertexes[length]:
                    chains = np.array([[0.,node]])
                    for layers in reversed(range(length)):
                        chain = copy(chains)
                        for nodes in vertexes[layers]:
                            for i in range(len(chain[:,-1])):
                                if chain[i][-1] in nodes.connections.keys():
                                    new_chain = np.append(chain[i], nodes)
                                    new_chain[0] = chains[i][0] + nodes.connections[chain[i][-1]]
                                    
                                    if len(chain[0]) == len(chains[0]):
                                        chains = np.hstack((chains, [[0] for k in range(chains.shape[0])]))
                                    chains = np.vstack((chains,new_chain))
                    for arr in chains:
                        temp = arr[1:][arr[1:] != 0]
                        if temp[0] == temp[-1] and len(temp) > 1:
                            cycles.add((arr[0],tuple(temp)))
                    break
        if verbose:
            for cycle in cycles:
                if cycle[0] > 0:
                    print('\nWeight = ', cycle[0])
                    print(list(map(lambda val: str(val), cycle[1])))
        return cycles
    
    
    def get_eigens(self):
        return np.linalg.eig(self.A)[0] 
    
    
    def stability(self):
        print(self.get_eigens())
        max_eigen = max(abs(self.get_eigens()))
        if max_eigen < 1:
            return True
        else:
            return False
    

In [ ]:
A = Node('A', 1)
C = Node('C', 2)
B = Node('B', 3)
D = Node('D', 4)

impulse = [np.array([[0], [0.1], [0.5]]) if i < 10 else np.array([[0], [0], [0]]) for i in range(50)]

A.set_connections([[0.8, C]])
B.set_connections([[0.2, B], [0.3, C], [0.111, A]])
C.set_connections([[0.2, C], [0, B], [0.3, A]])
Cognitive_Parasha = Graph('Parasha')
Cognitive_Parasha.set_nodes([A, B, C])
Cognitive_Parasha.form_subgraphs({'Bois': [A], 'Parasites': [B, C]})
Cognitive_Parasha.form_connection_matrix()
Cognitive_Parasha.search_cycles(verbose = True)
Cognitive_Parasha.get_eigens()
Cognitive_Parasha.stability()
#Cognitive_Parasha.send_impulses(impulse)

In [ ]:
nodes = []
for i in range(22):
    nodes.append(Node(str(i), i))

for node in nodes:
    node.set_connections([[1, neighbour] if neighbour != node else [1, nodes[-1]] for neighbour in nodes[: -1]])

full_graph = Graph('nodes')
full_graph.set_nodes(nodes)
#full_graph.search_cycles(verbose = True)

In [ ]:
max(abs(np.array([1,4,-5,3])))

In [ ]:
drawer = plotter.NetworkXPlotter(swot_graph)
drawer.plot(save_html=True, edge_color='Black', paper_bgcolor='White')


In [181]:
SWOT = swot.SWOT
swot_graph = Graph('swot')
swot_graph.from_pandas(SWOT, SW_table, 12)
swot_graph.form_connection_matrix()
swot_graph.stability()
swot_graph.search_cycles(verbose=True)

[ 2.85900846e+01 -9.97287628e+00 -5.85184014e+00 -5.55717579e+00
  4.55230816e+00 -4.55814649e+00  3.21476963e+00 -3.54710953e+00
  2.60197103e+00 -2.96205209e+00 -2.73438740e+00 -2.59085626e+00
  2.21787451e+00 -2.19425020e+00  2.02360730e+00 -1.89785794e+00
  1.65365050e+00 -1.59426798e+00 -1.46342138e+00 -1.27615855e+00
 -1.25478504e+00  1.16143264e+00 -1.00552953e+00 -9.30503691e-01
  1.05201062e+00 -7.51824701e-01  8.57329269e-01  8.26946660e-01
  7.22285091e-01 -5.13333663e-01  5.91236606e-01  4.74684136e-01
 -3.92720579e-01 -3.45837883e-01  3.67755818e-01 -2.19008844e-01
  2.85491485e-01 -7.04805143e-02  1.83114657e-01  1.46953229e-01
  1.09215258e-01  5.52607865e-02  1.13410361e-02 -1.48985263e-02]

Weight =  1.8
['Huge computational complexity', 'E-maps of road signs', 'Huge computational complexity']

Weight =  5.32
['No human interaction in driving', 'Rational route selection', 'No human interaction in driving']

Weight =  2.0
['Trolley problem', 'Rejection by people', 'Trol

['Prevention of car accidents', 'Job abolition', 'Prevention of car accidents']

Weight =  0.78
['Goverment support of autonomous cars dev', 'Man depends on vehicle', 'Goverment support of autonomous cars dev']

Weight =  1.22
['Speed limit', 'Prevention of car accidents', 'Speed limit']

Weight =  5.3
['Huge computational complexity', 'Rational route selection', 'Huge computational complexity']

Weight =  4.2
['System wear', 'Rational fuel usage', 'System wear']

Weight =  3.96
['No human interaction in driving', 'Huge computational complexity', 'No human interaction in driving']

Weight =  0.58
['ABS, ESP systems', 'Rational fuel usage', 'ABS, ESP systems']

Weight =  0.6
['Fully automated Traffic Regulations', 'Not ordinary behaivor of some road users', 'Fully automated Traffic Regulations']

Weight =  4.16
['Fully automated Traffic Regulations', 'Traffic analysis and lane moving control', 'Fully automated Traffic Regulations']

Weight =  0.96
['E-maps of road signs', 'Software bugs


Weight =  1.22
['GPS/Internet quality', 'Trolley problem', 'GPS/Internet quality']

Weight =  1.8
['Rational route selection', 'E-maps of road signs', 'Rational route selection']

Weight =  5.9
['Traffic analysis and lane moving control', 'No human interaction in driving', 'Traffic analysis and lane moving control']

Weight =  0.44
['Software bugs', 'Prevention of car accidents', 'Software bugs']

Weight =  2.02
['Software bugs', 'Fully automated Traffic Regulations', 'Software bugs']

Weight =  1.4
['Rational fuel usage', 'High demand on cars', 'Rational fuel usage']

Weight =  1.1
['Traffic analysis and lane moving control', 'Vehicle to vehicle connection', 'Traffic analysis and lane moving control']

Weight =  3.06
['Fully automated Traffic Regulations', 'High price', 'Fully automated Traffic Regulations']

Weight =  3.5
['Automatic photos', 'Huge computational complexity', 'Automatic photos']

Weight =  0.64
['ABS, ESP systems', 'Inadequacy of legal system', 'ABS, ESP systems']

W

{(1.8,
  (<__main__.Node at 0xa2c0f6780>,
   <__main__.Node at 0xa2c0f6780>)),
 (-0.1,
  (<__main__.Node at 0xa2c0f61d0>,
   <__main__.Node at 0xa2c0f61d0>)),
 (5.32,
  (<__main__.Node at 0xa2c0f69b0>,
   <__main__.Node at 0xa2c0f69b0>)),
 (2.0,
  (<__main__.Node at 0xa2c0f6278>,
   <__main__.Node at 0xa2c0f6278>)),
 (0.2,
  (<__main__.Node at 0xa2c0f6c88>,
   <__main__.Node at 0xa2c0f6c88>)),
 (0.74,
  (<__main__.Node at 0xa2c0ceba8>,
   <__main__.Node at 0xa2c0ceba8>)),
 (1.6,
  (<__main__.Node at 0xa2c0ce2e8>,
   <__main__.Node at 0xa2c0ce2e8>)),
 (2.12,
  (<__main__.Node at 0xa2c0f6828>,
   <__main__.Node at 0xa2c0f6828>)),
 (-2.2,
  (<__main__.Node at 0xa2c082a58>,
   <__main__.Node at 0xa2c082a58>)),
 (1.6,
  (<__main__.Node at 0xa2c0ce828>,
   <__main__.Node at 0xa2c0ce828>)),
 (1.5,
  (<__main__.Node at 0xa2c0ce940>,
   <__main__.Node at 0xa2c0ce940>)),
 (0.5,
  (<__main__.Node at 0xa2c0ceb38>,
   <__main__.Node at 0xa2c0ceb38>)),
 (1.0,
  (<__main__.Node at 0xa2c0ceb70>,
   <_

In [28]:
SWOT

,High production price,High service price,Not enough qualified mechanics,Bad road cover surface,Not ordinary behaivor of some road users,Rejection by people,"Hacking, confidentiality problems",Increased scam interest,Job abolition,Inadequacy of legal system,E-maps of road signs,Vehicle to vehicle connection,More passanger places (no driver needed),System to prevent trafic jams,Alternative energy usage,New powerful computer vision technologies,Cars technologies with less polution,High efficency engines,High demand on cars,Goverment support of autonomous cars dev
No human interaction in driving,0.00,0.40,0.00,0.40,0.80,0.00,0.75,0.75,0.0,0.00,0.50,0.00,0.99,0.86,0.00,0.41,0.80,0.80,0.90,0.65
Prevention of car accidents,0.09,0.30,0.50,0.70,0.90,0.95,0.00,0.20,0.5,0.40,0.40,0.20,0.80,0.97,0.00,0.00,0.00,0.00,0.85,0.95
Speed limit,0.40,0.40,0.40,0.60,0.60,0.00,0.00,0.00,0.0,0.50,0.80,0.40,0.60,0.90,0.30,0.31,0.69,0.50,0.10,0.80
Traffic analysis and lane moving control,0.00,0.40,0.10,0.45,1.00,0.80,0.00,0.00,0.0,0.20,0.90,0.55,0.80,0.85,0.20,0.70,0.54,0.20,0.60,0.75
"ABS, ESP systems",0.50,0.10,0.80,0.75,0.05,0.00,0.00,0.00,0.0,0.32,0.00,0.00,0.20,0.15,0.00,0.00,0.00,0.30,0.07,0.00
Automatic Parking,0.00,0.20,0.00,0.00,0.00,0.25,0.00,0.12,0.0,0.00,0.20,0.00,0.80,0.45,0.00,0.10,0.20,0.00,0.36,0.39
Fully automated Traffic Regulations,0.00,0.80,0.65,0.80,0.30,0.90,0.00,0.10,0.0,0.98,0.70,0.25,0.92,0.85,0.35,0.00,0.30,0.21,0.28,1.00
Rational route selection,0.00,0.00,0.00,0.50,0.00,0.17,0.40,0.40,0.3,0.00,0.90,0.25,0.90,1.00,0.28,0.00,0.75,0.20,0.65,0.00
Rational fuel usage,0.00,0.10,0.33,0.00,0.00,0.50,0.20,0.30,0.3,0.00,0.00,0.20,0.50,0.20,0.95,0.00,0.88,0.80,0.70,0.50
Public transport sync,0.00,0.00,0.00,0.10,0.50,0.80,0.00,0.00,0.0,0.28,0.30,0.70,0.93,0.10,0.20,0.00,0.00,0.00,0.00,0.80


In [142]:
SW_table = form_SW_table(SWOT, 10)

[ 0.    1.86  2.81  2.95  0.17  2.18  2.51  2.66  2.9   1.38  1.73  2.41
  0.46  1.98  1.69  1.    1.1   0.12  1.23 -0.03  0.9   1.4   1.97 -0.86]
